# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_pd = pd.read_csv("../WeatherPy/city_weather.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
cities_pd_copy = cities_pd.copy()
indexClouds = cities_pd_copy[cities_pd_copy["Cloudiness"] > 0].index
indexWind = cities_pd_copy[cities_pd_copy["Wind Speed"] >= 10].index
indexTemp = cities_pd_copy[(cities_pd_copy["Max Temperature"] <= 70) & 
                           (cities_pd_copy["Max Temperature"] > 79)].index

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = cities_pd[["Latitude", "Longitude"]].astype(float)
humidity_layer = gmaps.heatmap_layer(locations, weights=cities_pd["Humidity"])
humidity_layer.max_intensity = float(cities_pd["Humidity"].max())

fig = gmaps.figure()
fig.add_layer(humidity_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
cities_pd_copy = cities_pd.copy()
#indexClouds = cities_pd_copy[cities_pd_copy["Cloudiness"] > 0].index
#indexWind = cities_pd_copy[cities_pd_copy["Wind Speed"] >= 10].index
#indexTemp = cities_pd_copy[(cities_pd_copy["Max Temperature"] <= 70) & (cities_pd_copy["Max Temperature"] > 79)].index
indexIdeal = cities_pd_copy[(cities_pd_copy["Max Temperature"] <= 70)|(cities_pd_copy["Max Temperature"] > 79)|
                            (cities_pd_copy["Wind Speed"] >= 10)|(cities_pd_copy["Cloudiness"] > 0)|(cities_pd_copy["Humidity"] > 75)].index
#cities_pd_copy.drop(indexClouds, inplace=True)
#cities_pd_copy.drop(indexWind, inplace=True)
#cities_pd_copy.drop(indexTemp, inplace=True)
cities_pd_copy.drop(indexIdeal , inplace=True)
#print(indexIdeal)
cities_pd_copy = cities_pd_copy.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = cities_pd_copy
hotel_df["Hotel Name"]=""
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
219,219,Jardim,0,BR,1580011955,68,-21.48,-56.14,71.04,3.85,
288,288,San Rafael,0,AR,1580011955,28,-34.62,-68.33,77.09,2.57,
308,308,Jandaia do Sul,0,BR,1580011955,64,-23.60,-51.64,75.20,1.36,
381,381,Nicoya,0,CR,1580011955,61,10.15,-85.45,78.80,5.82,
383,383,Porto Murtinho,0,BR,1580011955,48,-21.70,-57.88,74.17,1.97,


In [15]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}


# Use the lat/lng we recovered to identify hotels
for index, row in cities_pd_copy.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    response = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
       print("Missing field/result... skipping.")
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
219,219,Jardim,0,BR,1580011955,68,-21.48,-56.14,71.04,3.85,Hotel Estância
288,288,San Rafael,0,AR,1580011955,28,-34.62,-68.33,77.09,2.57,Amukan Guest House
308,308,Jandaia do Sul,0,BR,1580011955,64,-23.60,-51.64,75.20,1.36,Hotel União
381,381,Nicoya,0,CR,1580011955,61,10.15,-85.45,78.80,5.82,Hotel Doña Marta
383,383,Porto Murtinho,0,BR,1580011955,48,-21.70,-57.88,74.17,1.97,Hotel Saladero Cue


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cities_pd_copy.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
# Add marker layer ontop of heat map
hotels = hotel_df["Hotel Name"].tolist()
cities = hotel_df["City"].tolist()
countries = hotel_df["Country"].tolist()


marker_layer = gmaps.symbol_layer(
    locations,
    info_box_content=hotel_info
    )
                      
    

# Display Map
fig = gmaps.figure()
fig.add_layer(humidity_layer)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))